In [1]:
import yaml

import bokeh
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Slider, Button
from bokeh.themes import Theme
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, output_file, Column
from bokeh.models import DataTable, TableColumn, PointDrawTool, ColumnDataSource
from bokeh.events import DoubleTap
from bokeh.models.widgets import TextInput

import im_click
import glob
import skimage.io

output_notebook()

Loading BokehJS ...

In [2]:
im_fs = glob.glob('../toy_data/ant_*.tiff')
ims = [skimage.io.imread(x) for x in im_fs]

In [3]:
point_labels = ColumnDataSource({'x': [],
                                 'y': [],
                                 'frame': []
                                })

alphas = []

pal = bokeh.palettes.viridis(len(ims))

def modify_doc(doc):
    im_num = [0,]
    
    images = ims.copy()
    plot, source = im_click.imshow(images[im_num[-1]], return_im=True)
    source = source.data_source
    
    renderer = plot.scatter(x='x', y='y', source=point_labels, color='white', size=3)
    columns = [TableColumn(field="x", title="x"),
               TableColumn(field="y", title="y"),
               TableColumn(field='frame', title='frame')]
    table = DataTable(source=point_labels, columns=columns, editable=True, height=200)
    draw_tool = PointDrawTool(renderers=[renderer], empty_value=im_num[-1])
    plot.add_tools(draw_tool)
    plot.toolbar.active_tap = draw_tool

    def callback(attr, old, new):
        im_num.append(int(new))
        _, data = im_click.imshow(images[int(new)], return_im=True)
        data = data.data_source
        source.data = data.data
        draw_tool.empty_value = im_num[-1]

    def callback_forward():
        if (len(images) - 2) < im_num[-1]:
            return None
        _, data = im_click.imshow(images[im_num[-1]+1], return_im=True)
        im_num.append(im_num[-1]+1)
        data = data.data_source
        source.data = data.data
        draw_tool.empty_value = im_num[-1]
        
    def callback_backward():
        if im_num[-1] == 0:
            return None
        _, data = im_click.imshow(images[im_num[-1]-1], return_im=True)
        im_num.append(im_num[-1]-1)
        data = data.data_source
        source.data = data.data
        draw_tool.empty_value = im_num[-1]
        
    slider = Slider(start=0, end=29, value=0, step=1, title="Frame Number")
    button_back = Button(label='back',button_type="success")
    button_forward = Button(label='forward',button_type="success")
    #text_input = TextInput(value="0", title="Frame Num:")
    #text_input.on_change('value', callback)
    slider.on_change('value', callback)
    button_forward.on_click(callback_forward)
    button_back.on_click(callback_backward)
    
    
    doc.add_root(column(row(slider), plot, row(button_back, button_forward), table))
    
show(modify_doc)

In [275]:
plot = figure()
renderer = plot.scatter(x='x', y='y', source=point_labels, color='white', size=3)